<a href="https://colab.research.google.com/github/WMadaraChamudini/PC_Assignment3/blob/main/IT23292154_PC_Assignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Matrix Multiplication-IT23292154-PC Assignment 03**

### **Serial Code**

In [1]:
%%writefile serial_mat_mul.c
//compile: gcc -O2 -std=c11 serial_mat_mul.c -o serial_mat_mul
//run: ./serial_mat_mul N  <-- N is the size of the matrix

#include <stdio.h>
#include <stdlib.h>
#include <time.h>

double* alloc_mat(int n){
  double *m =(double*) malloc((size_t) n*n*sizeof(double));
  if (!m){
    perror("malloc");
    exit(1);
  }
  return m;
}

void init_ex(double *A,double *B,int n){
  for (long i=0; i<(long)n*n; ++i){
    A[i]=(double)((i/n) + (i%n)+1);
    B[i]=(double)( ((i/n)+1) * ((i % n)+2) );
  }
}

void print_mat(const char *name, double *M, int n){
    printf("%s:\n", name);
    for (int i=0;i<n;++i) {
        for (int j=0;j<n;++j) {
            printf( "%8.2f ", M[(long)i*n + j] );
        }
        printf("\n");
    }
}

int main(int argc, char **argv){
  int n=3;

  if (argc>=2){
    n = atoi(argv[1]);
  }

  if (n<=0){
    printf("Invalid N\n");
    return 1;
  }

  double *A = alloc_mat(n);
  double *B = alloc_mat(n);
  double *C = alloc_mat(n);

  init_ex(A,B,n);
  for (long i=0; i<(long)n*n; ++i){
    C[i]=0.0;
  }

  clock_t t0 = clock();

  for (int i=0;i<n;++i) {
    for (int j=0;j<n;++j) {
      double sum = 0.0;
      for (int k=0;k<n;++k) sum += A[(long)i*n + k] * B[(long)k*n + j];
      C[(long)i*n + j] = sum;
    }
  }

  clock_t t1 = clock();

  //time for execution in seconds
  double Tot_t=(double) (t1-t0)/CLOCKS_PER_SEC;

  //Print the matrix if n<=10
  if (n<=10) {
    print_mat("Matrix A", A,n);
    printf("\n");
    print_mat("Matrix B", B,n);
    printf("\n");
    print_mat("Result Matrix C = A * B", C,n);
    printf("\n");
  }

  // checksum and per-row sums
  double checksum = 0.0;
  printf("Summary:\n");
  printf("Matrix size: %d x %d\n", n,n);
  printf("Execution time: %.6f seconds\n", Tot_t);
  for (int i=0;i<n;++i){
    double rowSum = 0.0;
    for (int j=0;j<n;++j){
      rowSum += C[(long)i*n + j];
    }
    printf(" Row %2d sum = %.2f\n", i, rowSum);
    checksum += rowSum;
  }
  printf("Checksum (sum of all elements) = %.6e\n", checksum);

  free(A);
  free(B);
  free(C);
  return 0;
}



Writing serial_mat_mul.c


In [2]:
!gcc serial_mat_mul.c -o serial_mat_mul

In [3]:
!./serial_mat_mul 4


Matrix A:
    1.00     2.00     3.00     4.00 
    2.00     3.00     4.00     5.00 
    3.00     4.00     5.00     6.00 
    4.00     5.00     6.00     7.00 

Matrix B:
    2.00     3.00     4.00     5.00 
    4.00     6.00     8.00    10.00 
    6.00     9.00    12.00    15.00 
    8.00    12.00    16.00    20.00 

Result Matrix C = A * B:
   60.00    90.00   120.00   150.00 
   80.00   120.00   160.00   200.00 
  100.00   150.00   200.00   250.00 
  120.00   180.00   240.00   300.00 

Summary:
Matrix size: 4 x 4
Execution time: 0.000002 seconds
 Row  0 sum = 420.00
 Row  1 sum = 560.00
 Row  2 sum = 700.00
 Row  3 sum = 840.00
Checksum (sum of all elements) = 2.520000e+03


In [4]:
!./serial_mat_mul 40

Summary:
Matrix size: 40 x 40
Execution time: 0.000359 seconds
 Row  0 sum = 19040400.00
 Row  1 sum = 19745600.00
 Row  2 sum = 20450800.00
 Row  3 sum = 21156000.00
 Row  4 sum = 21861200.00
 Row  5 sum = 22566400.00
 Row  6 sum = 23271600.00
 Row  7 sum = 23976800.00
 Row  8 sum = 24682000.00
 Row  9 sum = 25387200.00
 Row 10 sum = 26092400.00
 Row 11 sum = 26797600.00
 Row 12 sum = 27502800.00
 Row 13 sum = 28208000.00
 Row 14 sum = 28913200.00
 Row 15 sum = 29618400.00
 Row 16 sum = 30323600.00
 Row 17 sum = 31028800.00
 Row 18 sum = 31734000.00
 Row 19 sum = 32439200.00
 Row 20 sum = 33144400.00
 Row 21 sum = 33849600.00
 Row 22 sum = 34554800.00
 Row 23 sum = 35260000.00
 Row 24 sum = 35965200.00
 Row 25 sum = 36670400.00
 Row 26 sum = 37375600.00
 Row 27 sum = 38080800.00
 Row 28 sum = 38786000.00
 Row 29 sum = 39491200.00
 Row 30 sum = 40196400.00
 Row 31 sum = 40901600.00
 Row 32 sum = 41606800.00
 Row 33 sum = 42312000.00
 Row 34 sum = 43017200.00
 Row 35 sum = 43722400.00
 

### **OpenMP Code**

In [5]:
%%writefile openmp_mat_mul.c
// Compile: gcc -O3 -fopenmp openmp_mat_mul.c -o openmp_mat_mul
// Run: ./openmp_mat_mul N n <-- N is the size of the matrix, n is the number of threads

#include <stdio.h>
#include <stdlib.h>
#include <omp.h>

double* alloc_mat(int n){
  double *m = (double*) malloc((size_t)n*n*sizeof(double));
  if (!m){
    perror("malloc");
    exit(1);
  }
  return m;
}

void init_ex(double *A, double *B, int n){
  for (long i=0; i<(long)n*n; ++i){
    A[i] = (double) ((i/n)+(i%n)+1);
    B[i] = (double)(((i/n)+1) * ((i%n)+2));
  }
}

void print_mat(const char *name, double *M, int n){
  printf("%s:\n", name);
  for (int i=0; i<n; ++i){
    for (int j=0;j<n;++j){
      printf("%8.2f ", M [(long) i*n+j]);
    }
    printf("\n");
  }
}

int main(int argc, char **argv){
  if (argc<3){
    printf("Usage: %s N threads\n", argv[0]);
    return 1;
  }
  int n = atoi(argv[1]);
  int threads = atoi(argv[2]);
  if (n<=0 || threads<=0){
    printf("Invalid args\n");
    return 1;
  }

  omp_set_num_threads(threads);
  double *A = alloc_mat(n), *B = alloc_mat(n), *C = alloc_mat(n);
  init_ex(A,B,n);
  for (long i=0; i<(long)n*n; ++i){
    C[i]=0.0;
  }

 double t0 = omp_get_wtime();
 #pragma omp parallel
 {
  #pragma omp for schedule(static)
    for (int i=0; i<n; ++i){
      for (int k=0;k<n;++k){
        double a = A[(long)i*n + k];
        for (int j=0;j<n;++j) {
          C[(long)i*n + j] += a * B[(long)k*n + j];
        }
      }
    }
  }

  double t1 = omp_get_wtime();
  //time for execution in seconds
  double Tot_t = t1-t0;

  //Print the matrix if n<=10
  if (n<=10) {
    print_mat("Matrix A", A,n);
    printf("\n");
    print_mat("Matrix B", B,n);
    printf("\n");
    print_mat("Result Matrix C = A * B", C,n);
    printf("\n");
  }
  double checksum = 0.0;
  printf("Summary:\n");
  printf("Matrix size: %d x %d\n", n, n);
  printf("Threads used: %d\n", threads);
  printf("Execution time: %.6f seconds\n", Tot_t);
  for (int i=0; i<n; ++i){
    double rowSum=0.0;
    for (int j=0;j<n;++j){
      rowSum += C[(long)i*n+j];
    }
    printf("Row %2d sum = %.2f\n", i,rowSum);
    checksum+=rowSum;
  }
  printf("Checksum (sum of all elements) = %.6e\n", checksum);

  free(A); free(B); free(C);
  return 0;
}



Writing openmp_mat_mul.c


In [6]:
!gcc -O3 -fopenmp openmp_mat_mul.c -o openmp_mat_mul

In [7]:
!./openmp_mat_mul 4 4

Matrix A:
    1.00     2.00     3.00     4.00 
    2.00     3.00     4.00     5.00 
    3.00     4.00     5.00     6.00 
    4.00     5.00     6.00     7.00 

Matrix B:
    2.00     3.00     4.00     5.00 
    4.00     6.00     8.00    10.00 
    6.00     9.00    12.00    15.00 
    8.00    12.00    16.00    20.00 

Result Matrix C = A * B:
   60.00    90.00   120.00   150.00 
   80.00   120.00   160.00   200.00 
  100.00   150.00   200.00   250.00 
  120.00   180.00   240.00   300.00 

Summary:
Matrix size: 4 x 4
Threads used: 4
Execution time: 0.000238 seconds
Row  0 sum = 420.00
Row  1 sum = 560.00
Row  2 sum = 700.00
Row  3 sum = 840.00
Checksum (sum of all elements) = 2.520000e+03


In [8]:
!./openmp_mat_mul 40 4

Summary:
Matrix size: 40 x 40
Threads used: 4
Execution time: 0.000187 seconds
Row  0 sum = 19040400.00
Row  1 sum = 19745600.00
Row  2 sum = 20450800.00
Row  3 sum = 21156000.00
Row  4 sum = 21861200.00
Row  5 sum = 22566400.00
Row  6 sum = 23271600.00
Row  7 sum = 23976800.00
Row  8 sum = 24682000.00
Row  9 sum = 25387200.00
Row 10 sum = 26092400.00
Row 11 sum = 26797600.00
Row 12 sum = 27502800.00
Row 13 sum = 28208000.00
Row 14 sum = 28913200.00
Row 15 sum = 29618400.00
Row 16 sum = 30323600.00
Row 17 sum = 31028800.00
Row 18 sum = 31734000.00
Row 19 sum = 32439200.00
Row 20 sum = 33144400.00
Row 21 sum = 33849600.00
Row 22 sum = 34554800.00
Row 23 sum = 35260000.00
Row 24 sum = 35965200.00
Row 25 sum = 36670400.00
Row 26 sum = 37375600.00
Row 27 sum = 38080800.00
Row 28 sum = 38786000.00
Row 29 sum = 39491200.00
Row 30 sum = 40196400.00
Row 31 sum = 40901600.00
Row 32 sum = 41606800.00
Row 33 sum = 42312000.00
Row 34 sum = 43017200.00
Row 35 sum = 43722400.00
Row 36 sum = 44427600

In [10]:
!./openmp_mat_mul 40 8

Summary:
Matrix size: 40 x 40
Threads used: 8
Execution time: 0.000323 seconds
Row  0 sum = 19040400.00
Row  1 sum = 19745600.00
Row  2 sum = 20450800.00
Row  3 sum = 21156000.00
Row  4 sum = 21861200.00
Row  5 sum = 22566400.00
Row  6 sum = 23271600.00
Row  7 sum = 23976800.00
Row  8 sum = 24682000.00
Row  9 sum = 25387200.00
Row 10 sum = 26092400.00
Row 11 sum = 26797600.00
Row 12 sum = 27502800.00
Row 13 sum = 28208000.00
Row 14 sum = 28913200.00
Row 15 sum = 29618400.00
Row 16 sum = 30323600.00
Row 17 sum = 31028800.00
Row 18 sum = 31734000.00
Row 19 sum = 32439200.00
Row 20 sum = 33144400.00
Row 21 sum = 33849600.00
Row 22 sum = 34554800.00
Row 23 sum = 35260000.00
Row 24 sum = 35965200.00
Row 25 sum = 36670400.00
Row 26 sum = 37375600.00
Row 27 sum = 38080800.00
Row 28 sum = 38786000.00
Row 29 sum = 39491200.00
Row 30 sum = 40196400.00
Row 31 sum = 40901600.00
Row 32 sum = 41606800.00
Row 33 sum = 42312000.00
Row 34 sum = 43017200.00
Row 35 sum = 43722400.00
Row 36 sum = 44427600

### **MPI Code**

In [16]:
%%writefile mpi_mat_mul.c
// Compile: mpicc -O3 mpi_mat_mul.c -o mpi_mat_mul
// Run: mpirun --allow-run-as-root --oversubscribe -np P ./mpi_mat_mul N <-- P is the number of processes, N is the size of the matrix

#include <stdio.h>
#include <stdlib.h>
#include <mpi.h>

double* alloc_mat(int n){
  double *m = (double*) malloc((size_t)n*n*sizeof(double));
  if (!m){
    perror("malloc");
    exit(1);
  }
  return m;
}

void init_ex(double *A, double *B, int n){
  for (long i=0; i<(long)n*n; i++){
    A[i] = (double)( (i/n)+(i%n)+1 );
    B[i] = (double)( ((i/n)+1) * ((i%n)+2) );
  }
}

void print_mat(const char *name, double *M, int n){
  printf("%s:\n", name);
  for (int i=0; i<n; i++){
    for (int j=0; j<n; j++){
      printf("%8.2f ", M[i*n+j] );
    }
    printf("\n");
  }
}

int main(int argc, char **argv){
  MPI_Init(&argc, &argv);
  int rank, size;
  MPI_Comm_rank(MPI_COMM_WORLD, &rank);
  MPI_Comm_size(MPI_COMM_WORLD, &size);

  if (argc<2){
    if(rank==0){
      printf("Usage: %s N\n", argv[0]);
    }
    MPI_Finalize();
    return 1;
  }

  int n = atoi(argv[1]);
  if (n<=0){
    if(rank==0){
      printf("Invalid N\n");
    }
    MPI_Finalize();
    return 1;
  }

  double *A = NULL, *B = alloc_mat(n), *C = NULL;

  if (rank==0){
    A = alloc_mat(n);
    C = alloc_mat(n);
    init_ex(A,B,n);
  }

  //broadcast B to all processes
  MPI_Bcast(B, n*n, MPI_DOUBLE, 0, MPI_COMM_WORLD);

  //row distribution determination
  int rows_per_proc = n/size;
  int extra = n%size;
  int start = rank*rows_per_proc + (rank<extra ? rank:extra);
  int end = start + rows_per_proc + (rank<extra ? 1:0);
  int num_rows = end-start;

  double *A_rows = (double*) malloc(num_rows * n * sizeof(double));
  double *C_rows = (double*) malloc(num_rows * n * sizeof(double));

  // scatter rows of A
  if(rank==0){
    for(int r=0; r<size; r++){
      int r_start = r*rows_per_proc + (r<extra?r:extra);
      int r_end = r_start + rows_per_proc + (r<extra?1:0);
      int r_num = r_end - r_start;
      if(r==0){
        for(int i=0; i<r_num*n; i++){
          A_rows[i] = A[i];
        }
      }else{
        MPI_Send(A + r_start*n, r_num*n, MPI_DOUBLE, r, 0, MPI_COMM_WORLD);
      }
    }
  }else{
    MPI_Recv(A_rows, num_rows*n, MPI_DOUBLE, 0, 0, MPI_COMM_WORLD, MPI_STATUS_IGNORE);
  }

  //start time
  double t0 = MPI_Wtime();

  //compute local C
  for(int i=0; i<num_rows; i++){
    for(int j=0; j<n; j++){
      double sum = 0.0;
      for(int k=0; k<n; k++){
        sum += A_rows[i*n+k] * B[k*n+j];
      }
      C_rows[i*n + j] = sum;
    }
  }

  //end time
  double t1 = MPI_Wtime();

  //Total time
  double Tot_t = t1-t0;

  //gather results
  int *recvcounts = NULL;
  int *displs = NULL;
  if (rank==0){
    recvcounts = (int*) malloc(size * sizeof(int));
    displs = (int*) malloc(size * sizeof(int));
    int offset = 0;
    for(int r=0; r<size; r++){
      int r_start = r*rows_per_proc + (r<extra ? r:extra);
      int r_end = r_start + rows_per_proc + (r<extra ? 1:0);
      recvcounts[r] = (r_end-r_start)*n;
      displs[r] = offset;
      offset += recvcounts[r];
    }
  }

  MPI_Gatherv(C_rows, num_rows*n, MPI_DOUBLE, C, recvcounts, displs, MPI_DOUBLE, 0, MPI_COMM_WORLD);

  // reduce max execution time to root
  double max_time;
  MPI_Reduce(&Tot_t, &max_time, 1, MPI_DOUBLE, MPI_MAX, 0, MPI_COMM_WORLD);

  // Print results on root
  if(rank==0){
    if(n<=10){
      print_mat("Matrix A", A,n);
      printf("\n");
      print_mat("Matrix B", B,n);
      printf("\n");
      print_mat("Result Matrix C = A*B", C,n);
      printf("\n");
    }

    double checksum=0.0;
    printf("Summary:\n Matrix size: %d x %d\n", n,n);
    printf("Execution time(seconds): %.6f\n", max_time);
    for(int i=0; i<n; i++){
      double rowSum=0.0;
      for(int j=0; j<n; j++){
        rowSum += C[i*n + j];
      }
      printf(" Row %2d sum = %.2f\n", i,rowSum);
      checksum += rowSum;
    }
    printf("Checksum(sum of all elements)= %.6e\n", checksum);
  }

  free(B);
  free(A_rows);
  free(C_rows);
  if(rank==0){
    free(A);
    free(C);
    free(recvcounts);
    free(displs);
  }

  MPI_Finalize();
  return 0;
}


Overwriting mpi_mat_mul.c


In [17]:
!mpicc -O3 mpi_mat_mul.c -o mpi_mat_mul

In [18]:
!mpirun --allow-run-as-root --oversubscribe -np 4 ./mpi_mat_mul 4


Matrix A:
    1.00     2.00     3.00     4.00 
    2.00     3.00     4.00     5.00 
    3.00     4.00     5.00     6.00 
    4.00     5.00     6.00     7.00 

Matrix B:
    2.00     3.00     4.00     5.00 
    4.00     6.00     8.00    10.00 
    6.00     9.00    12.00    15.00 
    8.00    12.00    16.00    20.00 

Result Matrix C = A*B:
   60.00    90.00   120.00   150.00 
   80.00   120.00   160.00   200.00 
  100.00   150.00   200.00   250.00 
  120.00   180.00   240.00   300.00 

Summary:
 Matrix size: 4 x 4
Execution time(seconds): 0.000000
 Row  0 sum = 420.00
 Row  1 sum = 560.00
 Row  2 sum = 700.00
 Row  3 sum = 840.00
Checksum(sum of all elements)= 2.520000e+03


In [19]:
!mpirun --allow-run-as-root --oversubscribe -np 4 ./mpi_mat_mul 40


Summary:
 Matrix size: 40 x 40
Execution time(seconds): 0.000020
 Row  0 sum = 19040400.00
 Row  1 sum = 19745600.00
 Row  2 sum = 20450800.00
 Row  3 sum = 21156000.00
 Row  4 sum = 21861200.00
 Row  5 sum = 22566400.00
 Row  6 sum = 23271600.00
 Row  7 sum = 23976800.00
 Row  8 sum = 24682000.00
 Row  9 sum = 25387200.00
 Row 10 sum = 26092400.00
 Row 11 sum = 26797600.00
 Row 12 sum = 27502800.00
 Row 13 sum = 28208000.00
 Row 14 sum = 28913200.00
 Row 15 sum = 29618400.00
 Row 16 sum = 30323600.00
 Row 17 sum = 31028800.00
 Row 18 sum = 31734000.00
 Row 19 sum = 32439200.00
 Row 20 sum = 33144400.00
 Row 21 sum = 33849600.00
 Row 22 sum = 34554800.00
 Row 23 sum = 35260000.00
 Row 24 sum = 35965200.00
 Row 25 sum = 36670400.00
 Row 26 sum = 37375600.00
 Row 27 sum = 38080800.00
 Row 28 sum = 38786000.00
 Row 29 sum = 39491200.00
 Row 30 sum = 40196400.00
 Row 31 sum = 40901600.00
 Row 32 sum = 41606800.00
 Row 33 sum = 42312000.00
 Row 34 sum = 43017200.00
 Row 35 sum = 43722400.00

In [20]:
!mpirun --allow-run-as-root --oversubscribe -np 8 ./mpi_mat_mul 40


Summary:
 Matrix size: 40 x 40
Execution time(seconds): 0.000076
 Row  0 sum = 19040400.00
 Row  1 sum = 19745600.00
 Row  2 sum = 20450800.00
 Row  3 sum = 21156000.00
 Row  4 sum = 21861200.00
 Row  5 sum = 22566400.00
 Row  6 sum = 23271600.00
 Row  7 sum = 23976800.00
 Row  8 sum = 24682000.00
 Row  9 sum = 25387200.00
 Row 10 sum = 26092400.00
 Row 11 sum = 26797600.00
 Row 12 sum = 27502800.00
 Row 13 sum = 28208000.00
 Row 14 sum = 28913200.00
 Row 15 sum = 29618400.00
 Row 16 sum = 30323600.00
 Row 17 sum = 31028800.00
 Row 18 sum = 31734000.00
 Row 19 sum = 32439200.00
 Row 20 sum = 33144400.00
 Row 21 sum = 33849600.00
 Row 22 sum = 34554800.00
 Row 23 sum = 35260000.00
 Row 24 sum = 35965200.00
 Row 25 sum = 36670400.00
 Row 26 sum = 37375600.00
 Row 27 sum = 38080800.00
 Row 28 sum = 38786000.00
 Row 29 sum = 39491200.00
 Row 30 sum = 40196400.00
 Row 31 sum = 40901600.00
 Row 32 sum = 41606800.00
 Row 33 sum = 42312000.00
 Row 34 sum = 43017200.00
 Row 35 sum = 43722400.00

### **CUDA Code**

In [28]:
%%writefile cuda_mat_mul.cu
//Compile: nvcc -O3 -arch=sm_75   cuda_mat_mul.cu -o cuda_mat_mul
//Run: ./cuda_mat_mul N blockSize <-- N is the size of the matrix, blockSize is the block size

#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>

//macro for CUDA error checking
#define CHECK_CUDA_ERROR(val) check((val), #val, __FILE__, __LINE__)
void check(cudaError_t err, const char* const func, const char* const file, const int line){
  if (err!=cudaSuccess){
    fprintf(stderr, "CUDA Error at %s:%d in function %s: %s\n", file,line,func,cudaGetErrorString(err));
    exit(EXIT_FAILURE);
  }
}

float* alloc_mat(int n){
  float *m = (float*) malloc((size_t) n*n*sizeof(float));
  if (!m){
    perror("malloc");
    exit(1);
  }
  return m;
}

void init_ex(float *A, float *B, int n){
  for (long i=0; i<(long)n*n; i++){
    A[i] = (float)((i/n)+(i%n)+1);
    B[i] = (float)( ((i/n)+1) * ((i%n)+2) );
  }
}

void print_mat(const char *name, float *M, int n) {
  printf("%s:\n", name);
  for (int i=0; i<n; i++){
    for (int j=0; j<n; j++){
      printf("%8.2f ", M[i*n + j]);
    }
    printf("\n");
  }
}

//CUDA kernel for matrix multiplication
__global__ void matmul_kernel(float *A, float *B, float *C, int n){
  int row = blockIdx.y*blockDim.y +threadIdx.y;
  int col = blockIdx.x*blockDim.x +threadIdx.x;

  if (row<n && col<n){
    float sum = 0.0f;
    for (int k=0; k<n; k++){
      sum += A[row*n + k] * B[k*n + col];
    }
    C[row*n + col]=sum;
  }
}

int main(int argc, char **argv){
  if(argc<3){
    printf("Usage: %s N blockSize\n", argv[0]);
    return 1;
  }

  int n = atoi(argv[1]);
  int blockSize = atoi(argv[2]);
  if(n<=0||blockSize<=0){
    printf("Invalid arguments\n");
    return 1;
  }

  //allocate host matrices
  float *A = alloc_mat(n);
  float *B = alloc_mat(n);
  float *C = alloc_mat(n);

  init_ex(A,B,n);

  //initialize host C to zeros(to debugging comparison in kernel failures)
  for (long i=0; i<(long)n*n; i++){
    C[i] = 0.0f;
  }

  //Allocate device memory
  float *d_A, *d_B, *d_C;
  CHECK_CUDA_ERROR (cudaMalloc((void**)&d_A, n*n*sizeof(float)));
  CHECK_CUDA_ERROR (cudaMalloc((void**)&d_B, n*n*sizeof(float)));
  CHECK_CUDA_ERROR (cudaMalloc((void**)&d_C, n*n*sizeof(float)));

  //copy data to device
  CHECK_CUDA_ERROR (cudaMemcpy(d_A, A, n*n*sizeof(float), cudaMemcpyHostToDevice));
  CHECK_CUDA_ERROR (cudaMemcpy(d_B, B, n*n*sizeof(float), cudaMemcpyHostToDevice));
  //didn't copy host C to device <-- d_C is written to device by kernel

  //configure grid and block
  dim3 dimBlock(blockSize, blockSize);
  dim3 dimGrid( (n+blockSize-1)/blockSize, (n+blockSize-1)/blockSize);

  //CUDA events for timing
  cudaEvent_t start, stop;
  CHECK_CUDA_ERROR(cudaEventCreate(&start));
  CHECK_CUDA_ERROR(cudaEventCreate(&stop));
  CHECK_CUDA_ERROR(cudaEventRecord(start));

  //launch kernel
  matmul_kernel <<<dimGrid, dimBlock>>> (d_A, d_B, d_C, n);
  CHECK_CUDA_ERROR(cudaGetLastError()); //check errors from kernel launch

  //wait for kernel to finish
  CHECK_CUDA_ERROR(cudaDeviceSynchronize());

  CHECK_CUDA_ERROR(cudaEventRecord(stop));
  CHECK_CUDA_ERROR(cudaEventSynchronize(stop));

  float milliseconds=0;
  CHECK_CUDA_ERROR(cudaEventElapsedTime(&milliseconds, start, stop));

  //copy result back to host
    CHECK_CUDA_ERROR(cudaMemcpy(C, d_C, n*n*sizeof(float), cudaMemcpyDeviceToHost));

    // Print matrices if small
    if(n <= 10){
        print_mat("Matrix A", A, n); printf("\n");
        print_mat("Matrix B", B, n); printf("\n");
        print_mat("Result Matrix C = A * B", C, n); printf("\n");
    }

    // Compute checksum and row sums
    float checksum=0.0f;
    printf("Summary:\nMatrix size: %d x %d\n", n, n);
    printf("Execution time (ms): %.6f\n", milliseconds);
    for(int i=0;i<n;i++){
        float rowSum = 0.0f;
        for(int j=0;j<n;j++) rowSum += C[i*n + j];
        printf(" Row %2d sum = %.2f\n", i, rowSum);
        checksum += rowSum;
    }
    printf("Checksum (sum of all elements) = %.6e\n", checksum);

    // Free memory
    free(A); free(B); free(C);
    CHECK_CUDA_ERROR(cudaFree(d_A));
    CHECK_CUDA_ERROR(cudaFree(d_B));
    CHECK_CUDA_ERROR(cudaFree(d_C));
    CHECK_CUDA_ERROR(cudaEventDestroy(start));
    CHECK_CUDA_ERROR(cudaEventDestroy(stop));

    return 0;
}


Overwriting matmul_cuda_friendly.cu


In [28]:
%%writefile cuda_mat_mul.cu
// Compile: nvcc -O3 -arch=sm_75 cuda_mat_mul.cu -o cuda_mat_mul
// Run: ./cuda_mat_mul N blockSize <-- N is the size of the matrix,blockSize is the block size

#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h> //for runtime API functions

//macro for CUDA error checking
#define CHECK_CUDA_ERROR(val) check((val), #val, __FILE__, __LINE__)
void check(cudaError_t err, const char* const func, const char* const file, const int line){
  if (err!=cudaSuccess){
    fprintf (stderr, "CUDA Error at %s:%d in function %s: %s\n", file, line, func, cudaGetErrorString(err));
    exit(EXIT_FAILURE);
  }
}

float* alloc_mat(int n){
  float *m = (float*) malloc((size_t)n*n*sizeof(float));
  if (!m){
    perror("malloc");
    exit(1);
  }
  return m;
}

void init_ex(float *A, float *B, int n){
  for (long i=0; i<(long)n*n; i++){
    A[i] = (float)((i/n)+(i%n)+1);
    B[i] = (float)( ((i/n)+1) * ((i%n)+2));
  }
}

void print_mat(const char *name, float *M, int n){
  printf("%s:\n", name);
  for (int i=0; i<n; i++){
    for (int j=0; j<n; j++){
      printf("%8.2f ", M[i*n+j]);
    }
    printf("\n");
  }
}

//CUDA kernel for matrix multiplication
__global__ void matmul_kernel(float *A, float *B, float *C, int n){
  int row = blockIdx.y * blockDim.y + threadIdx.y;
  int col = blockIdx.x * blockDim.x + threadIdx.x;

  if (row<n && col<n){
    float sum = 0.0f;
    for (int k=0; k<n; k++){
      sum += A[row*n+k] * B[k*n+col];
    }
    C[row*n + col]=sum;
  }
}

int main(int argc, char **argv){
  if (argc<3){
    printf ("Usage: %s N blockSize\n", argv[0]);
    return 1;
  }

  int n = atoi(argv[1]);
  int blockSize = atoi(argv[2]);
  if (n<=0||blockSize<=0){
    printf("Invalid arguments\n");
    return 1;
  }

  //host matrices allocation
  float *A = alloc_mat(n);
  float *B = alloc_mat(n);
  float *C = alloc_mat(n);

  init_ex(A,B,n);

  //initialize host C to zeros (to debugging comparison in kernel failures)
  for (long i=0; i<(long)n*n; i++){
    C[i] = 0.0f;
  }

  //device memory allocation
  float *d_A, *d_B, *d_C;
  CHECK_CUDA_ERROR(cudaMalloc((void**)&d_A, n*n*sizeof(float)));
  CHECK_CUDA_ERROR(cudaMalloc((void**)&d_B, n*n*sizeof(float)));
  CHECK_CUDA_ERROR(cudaMalloc((void**)&d_C, n*n*sizeof(float)));

  // Copy data -> device
  CHECK_CUDA_ERROR(cudaMemcpy(d_A, A, n*n*sizeof(float), cudaMemcpyHostToDevice));
  CHECK_CUDA_ERROR(cudaMemcpy(d_B, B, n*n*sizeof(float), cudaMemcpyHostToDevice));
  //didn't copy host C to device <-- d_C is written to device by kernel

  //configure grid and block
  dim3 dimBlock(blockSize, blockSize);
  dim3 dimGrid( (n+blockSize-1)/blockSize, (n+blockSize-1)/blockSize);

  //CUDA events for timing
  cudaEvent_t start, stop;
  CHECK_CUDA_ERROR(cudaEventCreate(&start));
  CHECK_CUDA_ERROR(cudaEventCreate(&stop));
  CHECK_CUDA_ERROR(cudaEventRecord(start));

  // Launch kernel
  matmul_kernel <<<dimGrid, dimBlock>>> (d_A, d_B, d_C, n);
  CHECK_CUDA_ERROR(cudaGetLastError()); //check for errors from kernel launch

  //wait for kernel to finish
  CHECK_CUDA_ERROR(cudaDeviceSynchronize());

  CHECK_CUDA_ERROR(cudaEventRecord(stop));
  CHECK_CUDA_ERROR(cudaEventSynchronize(stop));

  float milliseconds = 0;
  CHECK_CUDA_ERROR(cudaEventElapsedTime (&milliseconds, start, stop));
  float seconds = milliseconds/1000.0f; //convert milliseconds to seconds

  //copy result back to host
  CHECK_CUDA_ERROR(cudaMemcpy(C, d_C, n*n*sizeof(float), cudaMemcpyDeviceToHost));

  //print matrices(when N<10)
  if (n<=10){
    print_mat("Matrix A", A,n);
    printf("\n");
    print_mat("Matrix B", B,n);
    printf("\n");
    print_mat("Result Matrix C = A * B", C,n);
    printf("\n");
  }

  //compute checksum and row sums
  float checksum = 0.0f;
  printf("Summary:\nMatrix size: %d x %d \n", n,n);
  printf("Execution time(seconds): %.6f\n", seconds);
  for(int i=0; i<n; i++){
    float rowSum = 0.0f;
    for (int j=0; j<n; j++){
      rowSum += C[i*n + j];
    }
    printf(" Row %2d sum = %.2f\n", i,rowSum);
    checksum += rowSum;
  }
  printf("Checksum (sum of all elements) = %.6e\n", checksum);

  //free memory
  free(A);
  free(B);
  free(C);
  CHECK_CUDA_ERROR(cudaFree(d_A));
  CHECK_CUDA_ERROR(cudaFree(d_B));
  CHECK_CUDA_ERROR(cudaFree(d_C));
  CHECK_CUDA_ERROR(cudaEventDestroy(start));
  CHECK_CUDA_ERROR(cudaEventDestroy(stop));

  return 0;
}


Overwriting cuda_mat_mul.cu


In [29]:
!nvcc -O3 -arch=sm_75 cuda_mat_mul.cu -o cuda_mat_mul

In [30]:
!./cuda_mat_mul 4 4

Matrix A:
    1.00     2.00     3.00     4.00 
    2.00     3.00     4.00     5.00 
    3.00     4.00     5.00     6.00 
    4.00     5.00     6.00     7.00 

Matrix B:
    2.00     3.00     4.00     5.00 
    4.00     6.00     8.00    10.00 
    6.00     9.00    12.00    15.00 
    8.00    12.00    16.00    20.00 

Result Matrix C = A * B:
   60.00    90.00   120.00   150.00 
   80.00   120.00   160.00   200.00 
  100.00   150.00   200.00   250.00 
  120.00   180.00   240.00   300.00 

Summary:
Matrix size: 4 x 4 
Execution time(seconds): 0.000106
 Row  0 sum = 420.00
 Row  1 sum = 560.00
 Row  2 sum = 700.00
 Row  3 sum = 840.00
Checksum (sum of all elements) = 2.520000e+03


In [32]:
!./cuda_mat_mul 40 4

Summary:
Matrix size: 40 x 40 
Execution time(seconds): 0.000108
 Row  0 sum = 19040400.00
 Row  1 sum = 19745600.00
 Row  2 sum = 20450800.00
 Row  3 sum = 21156000.00
 Row  4 sum = 21861200.00
 Row  5 sum = 22566400.00
 Row  6 sum = 23271600.00
 Row  7 sum = 23976800.00
 Row  8 sum = 24682000.00
 Row  9 sum = 25387200.00
 Row 10 sum = 26092400.00
 Row 11 sum = 26797600.00
 Row 12 sum = 27502800.00
 Row 13 sum = 28208000.00
 Row 14 sum = 28913200.00
 Row 15 sum = 29618400.00
 Row 16 sum = 30323600.00
 Row 17 sum = 31028800.00
 Row 18 sum = 31734000.00
 Row 19 sum = 32439200.00
 Row 20 sum = 33144400.00
 Row 21 sum = 33849600.00
 Row 22 sum = 34554800.00
 Row 23 sum = 35260000.00
 Row 24 sum = 35965200.00
 Row 25 sum = 36670400.00
 Row 26 sum = 37375600.00
 Row 27 sum = 38080800.00
 Row 28 sum = 38786000.00
 Row 29 sum = 39491200.00
 Row 30 sum = 40196400.00
 Row 31 sum = 40901600.00
 Row 32 sum = 41606800.00
 Row 33 sum = 42312000.00
 Row 34 sum = 43017200.00
 Row 35 sum = 43722400.00

In [33]:
!./cuda_mat_mul 40 8

Summary:
Matrix size: 40 x 40 
Execution time(seconds): 0.000101
 Row  0 sum = 19040400.00
 Row  1 sum = 19745600.00
 Row  2 sum = 20450800.00
 Row  3 sum = 21156000.00
 Row  4 sum = 21861200.00
 Row  5 sum = 22566400.00
 Row  6 sum = 23271600.00
 Row  7 sum = 23976800.00
 Row  8 sum = 24682000.00
 Row  9 sum = 25387200.00
 Row 10 sum = 26092400.00
 Row 11 sum = 26797600.00
 Row 12 sum = 27502800.00
 Row 13 sum = 28208000.00
 Row 14 sum = 28913200.00
 Row 15 sum = 29618400.00
 Row 16 sum = 30323600.00
 Row 17 sum = 31028800.00
 Row 18 sum = 31734000.00
 Row 19 sum = 32439200.00
 Row 20 sum = 33144400.00
 Row 21 sum = 33849600.00
 Row 22 sum = 34554800.00
 Row 23 sum = 35260000.00
 Row 24 sum = 35965200.00
 Row 25 sum = 36670400.00
 Row 26 sum = 37375600.00
 Row 27 sum = 38080800.00
 Row 28 sum = 38786000.00
 Row 29 sum = 39491200.00
 Row 30 sum = 40196400.00
 Row 31 sum = 40901600.00
 Row 32 sum = 41606800.00
 Row 33 sum = 42312000.00
 Row 34 sum = 43017200.00
 Row 35 sum = 43722400.00